In [2]:
import pymongo
from pymongo import MongoClient
from gensim import corpora, models, similarities
from gensim.models import Word2Vec as w2v
from collections import defaultdict
import jieba
import io
import pymongo
from pymongo import MongoClient
import json
import jieba
from gensim import corpora, models, similarities
from collections import defaultdict
import jieba
import io

stoplist = []
documents =[]
words=[]
client = MongoClient() #預設連接自己主機上的mongodb
database = client["test"] # SQL: Database Name
collection = database["test"]
jieba.set_dictionary('dict1.txt')
jieba.load_userdict("dict_keyw_new.txt") 
jieba.load_userdict("dict_cbdic.txt") 
with io.open('C:/Users/BIG DATA/Desktop/stoplist.txt','r',encoding = 'utf-8') as f:
    for line in f.readlines():
        stoplist.append(line.strip('\n'))
for post in collection.find({"hitcount":{"$gt":10000}}):
    documents.append(post['content'])

for doc in range(0,len(documents)):
    word=[]
    z=unicode(documents[doc])
    for a in jieba.cut(z):
        if not a in stoplist:
            
            ab=[a]
            word+=ab
    #print word
    words+=[word]
model = w2v(words, min_count=1, size=100)

Building prefix dict from C:\Users\BIG DATA\dict1.txt ...
Loading model from cache c:\users\bigdat~1\appdata\local\temp\jieba.ueb82094ea19eedbe1ac558ddc779bc67.cache
Loading model cost 0.325 seconds.
Prefix dict has been built succesfully.


In [3]:
similar=model.most_similar(u'柯文哲',topn=30)
for a in similar:
    print a[0],a[1]

市長 0.981638252735
郝龍斌 0.978409349918
金溥聰 0.975771903992
王世堅 0.973670959473
徐弘庭 0.972099006176
廖國棟 0.970182776451
應訊 0.969488501549
柯建銘 0.969385802746
梁文傑 0.968420386314
高婉倩 0.968289494514
民進 0.967409670353
羅淑蕾 0.967247247696
王鴻薇 0.967159450054
重話 0.967016577721
張善政 0.967012166977
許福明 0.966319203377
黨內人士 0.966039180756
蔡正元 0.965552091599
議員 0.965519249439
國慶大會 0.965085268021
郁慕明 0.964359104633
艱困選區 0.963997483253
有人問 0.963585078716
王如玄 0.963286876678
世勳 0.962693154812
陳建仁 0.962139904499
楊家俍 0.962072849274
李鴻源 0.962039768696
黨中央 0.961690962315
叩應 0.961489081383


In [4]:
def W2VClassification(classes,corpus,similarLevel):
    dicT = {}   #類別字典
    Expt = []   #無法被W2V辨識的
    Remain = [] #低於門檻值的
    cnt = [] #計算長度用的陣列
    
    for a in classes:  #把類別名稱,當作Key,加入字典
        dicT[a] = set() #把每個Key的Value設為set ( 避免重複 )
        
    for q in open(corpus,'r'):   #把萃取關鍵字的結果讀出來
        q = q.strip().decode('utf-8')   #去空白,解碼
        cnt.append(q)  #把每一行都加進去-->用來算長度
        x1 = []
        try:
            for u in classes:  #從自訂好的類別一個個去跟'每個關鍵字'比對相似度
                x1.append(float(model.similarity(q,u.decode('utf-8')))) #把相似度寫進x1理面
            for i in range(0,len(classes)): #從 1~ 類別群數量
                if x1[i] == sorted(x1,reverse = True)[0]:  #後者:排序過後的第一個(相似度最高的);
                                                           #前者:遍歷每一個x1
                                                           #也就是問,這個關鍵字跟哪個類別最像
                    if x1[i] >= similarLevel: #設定相似門檻,過濾相似度過低的
                        dicT[classes[i]].add(q) #把相似度最高的加進字典的該類別裡
                    else:
                        Remain.append(q) #沒過門檻值的
        except:
            Expt.append(q) #W2V不認識的
            
    #把原長度cnt,跟去重複長度(轉成set)都印出來
    print '原總量',len(cnt),'\t|','去重複後',len(set(cnt))
    print '--------------------------------------------------'
    
    #把分好類的每一個類別內容印出來
    Scnt = 0
    for w in dicT:
        print '【',w,'】','\t數量:',len(dicT[w])
        for i in dicT[w]:
            print i,
            Scnt += 1
        print '\n'

    print '--------------------------------------------------'
    print '相似度門檻',similarLevel,'\t|','成功分類',Scnt,'\t|','成功分類比率',int(float(Scnt)/float(len(set(cnt)))*100),'%'
    print '--------------------------------------------------\n'

    #在此之前Expt 和 Remain都是List , 都有照順序排好 , 印出來可檢查這些詞的原順序
    #此時把List轉成Set , 去除重複值 , 增加可讀性
    Expt = set(Expt)
    Remain = set(Remain)
    
    #把W2V不認識的印出來
    print '【W2V無法辨識】\t數量',len(Expt)
    for r in Expt:
        print r,
    print '\n'
    
    #把沒過門檻值的印出來
    print '【低於門檻值】\t數量',len(Remain)
    for r in Remain:
        print r,
    print '\n'

In [7]:
classes = ['國際','社會','體育','政治','生活','股票','蔡英文','林書豪']
W2VClassification(classes,'dict_keyw_new.txt',0.2)

原總量 95656 	| 去重複後 95436
--------------------------------------------------
【 蔡英文 】 	數量: 1115
極限挑戰 馬習會 摸彩 蔡同榮 勞爾卡斯楚 林錫耀 補充水分 落選 悶經濟 立委蔡其昌 紀念儀式 候選人名單 壯志未酬 通過決議 會說話 沉靜 蘇起 選舉時 下屆 準閣揆 擔任高雄 中央委員 力挺 何煖軒 陳建仁 廣電三法 廿九名 虛心檢討 專訪 改革方案 施崇棠 治安會報 中常會 讚聲 周亭羽 夜衝 大陸國家 輔選 盧嘉辰 多數黨 范麗青 官銜 副總統 賽珍珠 國民黨立委 藍營立委 布蘭特 國大黨 民主黨籍 黨章 質詢 內鬨 國會 亞投行 做一半 人小組 媽祖婆 黨庫 民進黨 立委蔡錦隆 羅紹和 關說案 柯男 向選民 張天 炮製 共和黨 不分區立委 看望 瑞信 退選 下鄉 來台打工 蘇嘉全 蔡正元 卅位 補助申請 國安會議 否決 雷倩 房務員 黨營事業 行腳 陳以信 陳惠敏 國民黨主席 同額競選 交接條例 鍾小平 委員 拍MV 提案權 最綠 大陸台商 撤案 星雲法師 查賄 民進黨要求 黨籍立委 蔡英文 穿衣風格 洗刷 綠營 過半 黨機器 市府高層 林祖儀 申請登記 青年政策 辦尾牙 有給職顧問 松樹 德政 底牌 就職演說 宣示主權 人領表 陳節如 長陳菊 公開 八德路 民進黨立委 王進士 祕魯 常務副會長 太急 許歷農 台企聯 首長會議 全國政協 角逐 馬友友 海區 赴德 勞委會 終身學習 入會 國會改革 黨版 預算審查 當選人 施藍旗 新型專利 四本 瘦肉精美豬 陳明文 穩拿 飛碟電台 催討 海芬 自許 左派 老鳥 何瑞英 特別費 柯陣營 執政 答詢時 積功德 副總統選舉 加薪四法 修憲 換柱 守夜 未婚男女 複試 國親合 楊偉中 老柯 訪賓 抨擊 殭屍 美國國防部 段宜康 教育基金會 九年間 香菇肉羹 林國慶 立法院 攤子 人事布局 出席 完全執政 台大校友 參選黨主席 選情告急 李桐豪 一百五十萬 主委 內閣制 訴願會 國防 總統參選人 紀念特展 任內 領先達 選戰 九月政爭 醉毆運將案 陳情案 選舉 向林 拿不出政績 楊瓊瓔 李登輝 衛生部門 府方 鳳山國中 提名作業 蔡依珊 政見 楊秀慧 台北市長 國家歌劇院 首位女性 中國國家 前置作業 退黨 選民結構 鄭正鈐 樹黨 王